# environment setting

In [ ]:
! pip install datasets transformers rouge-score nltk

     |████████████████████████████████| 311 kB 9.4 MB/s 
     |████████████████████████████████| 3.5 MB 44.4 MB/s 
     |████████████████████████████████| 1.1 MB 58.9 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 133 kB 46.4 MB/s 
     |████████████████████████████████| 243 kB 58.1 MB/s 
     |████████████████████████████████| 596 kB 56.5 MB/s 
     |████████████████████████████████| 6.8 MB 46.3 MB/s 
     |████████████████████████████████| 895 kB 65.1 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 271 kB 50.5 MB/s 
     |████████████████████████████████| 144 kB 57.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, store the authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join)) then execute the following cell and input your username and password:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


install Git-LFS. Uncomment the following instructions:

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-1
  cuda-nsight-systems-10-1 cuda-nsight-systems-

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

4.16.2


# Fine-tuning a model on a summarization task

In [ ]:
model_checkpoint = "t5-base"

model checkpoint from the [Model Hub](https://huggingface.co/models). Here we picked the [`t5-base`](https://huggingface.co/t5-base) checkpoint. 

## Loading the dataset

use the [Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [ ]:
from datasets import load_dataset, load_metric

metric = load_metric("rouge")

Downloading:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

Load json files to /content folder from summary_NER/summary_data/SciTLDR-AIC folder





In [ ]:
from datasets import DatasetDict
train = DatasetDict.from_json('train.jsonl')
dev = DatasetDict.from_json('dev.jsonl')
test = DatasetDict.from_json('test.jsonl')

Using custom data configuration default-ed0564b8e9f50674


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-8c7539ff7544f4e8


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-ed0564b8e9f50674/0.0.0. Subsequent calls will reuse this data.


...

Using custom data configuration default-5bb66e74c7ccf76c


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-8c7539ff7544f4e8/0.0.0. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-5bb66e74c7ccf76c/0.0.0. Subsequent calls will reuse this data.


In [ ]:
doc_train = [ ' '.join(t['source']) for t in train ]
doc_test = [ ' '.join(t['source']) for t in test ]
doc_dev = [ ' '.join(t['source']) for t in dev ]

doc_test

In [ ]:
label_train = [t['target'][0] for t in train]
label_test = [t['target'][0] for t in test]
label_dev = [t['target'][0] for t in dev]
label_train

In [ ]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import pandas as pd
toks_train = [ word_tokenize(t) for t in doc_train ]
toks_test = [ word_tokenize(t) for t in doc_test ]
toks_dev = [ word_tokenize(t) for t in doc_dev ]
train_doc_len = pd.Series([len(x) for x in toks_train])
test_doc_len = pd.Series([len(x) for x in toks_test])
dev_doc_len = pd.Series([len(x) for x in toks_dev])

In [ ]:
train_doc_len[0]

1225

In [ ]:
toks_label_train = [word_tokenize(t) for t in label_train]
toks_label_test = [word_tokenize(t) for t in label_test]
toks_label_dev = [word_tokenize(t) for t in label_dev]
train_label_len = pd.Series([len(x) for x in toks_label_train])
test_label_len = pd.Series([len(x) for x in toks_label_test])
dev_label_len = pd.Series([len(x) for x in toks_label_dev])


Some statistics of number of words of paper in dataset

In [ ]:
train_label_len.describe()  # train max:119

count    1992.000000
mean       20.828815
std         8.634047
min         3.000000
25%        15.000000
50%        20.000000
75%        26.000000
max       119.000000
dtype: float64

In [ ]:
train_doc_len.describe()

count    1992.000000
mean     1124.068273
std       507.610838
min        41.000000
25%       808.750000
50%      1086.000000
75%      1391.250000
max      3687.000000
dtype: float64

In [ ]:
train = train.add_column('text',doc_train)
test = test.add_column('text',doc_test)
dev = dev.add_column('text',doc_dev)

In [ ]:
train = train.add_column('tldr',label_train)
test = test.add_column('tldr',label_test)
dev = dev.add_column('tldr',label_dev)

In [ ]:
test[2]['tldr']

'We show how discrete objects can be learnt in an unsupervised fashion from pixels, and how to perform reinforcement learning using this object representation.'

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [ ]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each predictions
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_agregator: Return aggregates if this is set to True
Retu

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings:

In [ ]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that the model requires.

Instantiate a tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- get a tokenizer that corresponds to the model architecture we want to use,
- download the vocabulary used when pretraining this specific checkpoint.


In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
len(tokenizer('summarize: '+train[0]['text'],max_length=1024, truncation=True)['input_ids'])

1024

In [ ]:
tokenizer('summarize: '+train[0]['text'],max_length=2048, truncation=True)

{'input_ids': [21603, 10, 6984, 12, 8, 1269, 13, 1659, 1036, 12, 11795, 3, 9, 1196, 13, 4421, 1437, 1036, 4145, 6, 132, 19, 3, 9, 6937, 1046, 16, 1705, 1453, 3621, 21, 761, 24228, 5275, 45, 3, 9, 13605, 2663, 5, 27989, 120, 6, 8, 2605, 13, 2404, 979, 11, 8, 3283, 300, 135, 33, 13, 3172, 12, 2082, 8, 31098, 821, 13, 11295, 16783, 5, 86, 48, 1040, 6, 62, 370, 3, 9, 1316, 11, 6684, 3, 28561, 13, 8, 18355, 2807, 21, 8, 2404, 979, 41, 9, 7, 168, 38, 1252, 10558, 52, 7, 61, 13, 8, 2812, 1453, 3621, 21, 13080, 24228, 5275, 5, 101, 504, 24, 8, 18355, 2807, 13, 8, 2404, 979, 1848, 1737, 8, 2620, 13, 8, 3, 9921, 1453, 3621, 38, 168, 38, 8, 1316, 11, 6684, 1124, 12, 1984, 1252, 2559, 5, 7053, 6, 62, 9248, 8, 18355, 2807, 13, 8, 2404, 979, 12, 1848, 1737, 8, 3283, 2605, 21, 8, 1453, 3621, 13, 13080, 24228, 5275, 11, 16906, 419, 9138, 5275, 5, 555, 1090, 7489, 19, 24, 10, 818, 8, 1453, 1681, 13, 13080, 5275, 65, 150, 18421, 2936, 415, 2559, 6, 8, 1453, 1681, 13, 80, 18, 11740, 537, 18, 18270, 529, 

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Instead of one sentence, we can pass along a list of sentences:

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

To prepare the targets for our model, we need to tokenize them inside the `as_target_tokenizer` context manager. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


prefix the inputs with "summarize:" (the model can also translate and it needs the prefix to know which task it has to perform).

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

 write the function that will preprocess samples. Just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so pad examples to the longest length in the batch and not the whole dataset.

In [ ]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [ prefix+x for x in examples['text'] ]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["tldr"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
len(preprocess_function(train[0])['input_ids'])

7484

This will apply the function on all the elements of all the splits in `dataset`, so training, validation and testing data will be preprocessed in one single command.

In [ ]:
dataset_train = train.map(preprocess_function, batched=True)
dataset_test = test.map(preprocess_function, batched=True)
dataset_dev = dev.map(preprocess_function, batched=True)


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

To instantiate a `Seq2SeqTrainer`, we will need to define three more things. The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
batch_size = 2
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-scitldr",
    evaluation_strategy = "epoch",
    learning_rate=3e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions.D efine a function for this, which will just use the `metric` we loaded earlier

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=dataset_train,
    eval_dataset=dataset_dev,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/t5-base-finetuned-scitldr is already a clone of https://huggingface.co/HenryHXR/t5-base-finetuned-scitldr. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp half precision backend


We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, paper_id, rouge_scores, source_labels, title, tldr, ic, source, target.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1992
  Num Epochs = 2
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 1992


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.053300,2.028496,34.977400,16.616300,30.617700,30.703800,18.798100
2,2.099400,2.023211,35.213400,16.891900,30.844200,30.931600,18.798100


Saving model checkpoint to t5-base-finetuned-scitldr/checkpoint-500
Configuration saved in t5-base-finetuned-scitldr/checkpoint-500/config.json
Model weights saved in t5-base-finetuned-scitldr/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-base-finetuned-scitldr/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-base-finetuned-scitldr/checkpoint-500/special_tokens_map.json
tokenizer config file saved in t5-base-finetuned-scitldr/tokenizer_config.json
Special tokens file saved in t5-base-finetuned-scitldr/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, paper_id, rouge_scores, source_labels, title, tldr, ic, source, target.
***** Running Evaluation *****
  Num examples = 619
  Batch size = 2
Saving model checkpoint to t5-base-finetuned-scitldr/checkpoint-1000
Configuration saved in t5-base-finetuned-scitldr/checkpoint-1000/con

TrainOutput(global_step=1992, training_loss=2.075509190080635, metrics={'train_runtime': 1521.7695, 'train_samples_per_second': 2.618, 'train_steps_per_second': 1.309, 'total_flos': 4810888651253760.0, 'train_loss': 2.075509190080635, 'epoch': 2.0})

upload the result of the training to the Hub

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to t5-base-finetuned-scitldr
Configuration saved in t5-base-finetuned-scitldr/config.json
Model weights saved in t5-base-finetuned-scitldr/pytorch_model.bin
tokenizer config file saved in t5-base-finetuned-scitldr/tokenizer_config.json
Special tokens file saved in t5-base-finetuned-scitldr/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.38k/850M [00:00<?, ?B/s]

Upload file runs/Feb05_05-10-17_c7dade86fb41/events.out.tfevents.1644037829.c7dade86fb41.80.10:  58%|#####7   …

To https://huggingface.co/HenryHXR/t5-base-finetuned-scitldr
   4678fdc..9336e32  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Rouge1', 'type': 'rouge', 'value': 35.2134}]}
To https://huggingface.co/HenryHXR/t5-base-finetuned-scitldr
   9336e32..c475ada  main -> main



'https://huggingface.co/HenryHXR/t5-base-finetuned-scitldr/commit/9336e32a2e29d3a1d97c201ba080fbc14d598b51'

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```

## Model inference

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("HenryHXR/t5-base-finetuned-scitldr")

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("HenryHXR/t5-base-finetuned-scitldr")

Downloading:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

In [ ]:
ARTICLE = test[0]['text']
inputs = tokenizer("summarize: " + ARTICLE, return_tensors="pt", max_length=1024, truncation=True)
outputs = model.generate(
    inputs["input_ids"], max_length=128, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
)

print(tokenizer.decode(outputs[0]))

<pad> We propose FearNet, a brain-inspired system for incremental class learning that significantly outperforms previous methods for catastrophic forgetting in deep neural networks. FearNet uses a brain-inspired dual-memory system in which new memories are consolidated from a network for recent memories inspired by the mammalian hippocampal complex to a network for long-term storage inspired by medial prefrontal cortex.</s>


In [ ]:
len(tokenizer.decode(outputs[0]))
len(train)

1992